In [27]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows',500)


![CRISP_DM](Crisp_DM_Tasks.png)

# Group-by apply on large (relational) data set

In [28]:
pd_JH_data=pd.read_csv('C:/ProgramData/Anaconda3/eps_covid19/data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,"Korea, South",1
2,2020-01-22,no,Kosovo,0
3,2020-01-22,no,Kuwait,0
4,2020-01-22,no,Kyrgyzstan,0


In [29]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-07-30')]

In [30]:
test_data#.head()

,date,state,country,confirmed
50849,2020-07-31,no,Germany,210399
50889,2020-07-31,no,US,4562107
51115,2020-08-01,no,Germany,211005
51155,2020-08-01,no,US,4620592
51381,2020-08-02,no,Germany,211220
51420,2020-08-02,no,US,4668172
51647,2020-08-03,no,Germany,212111
51687,2020-08-03,no,US,4713540
51912,2020-08-04,no,Germany,212828
51952,2020-08-04,no,US,4771080


In [31]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-08-25,no,237583
US,2020-08-25,no,5777710


In [32]:
# %load C:/ProgramData/Anaconda3/eps_covid19/src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate
        Parameters:
        ----------
        in_array : pandas.series
        Returns:
        ----------
        Doubling rate: double
    '''

    #print(in_array)
    #print(type)
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    #y = np.array(in_array['confirmed'])
    #X = np.arange(len(y)).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))


the test slope is: [2.]


In [33]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-08-25     237583
      US      2020-08-25    5777710

In [34]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [35]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [36]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  50849           NaN
                51115           NaN
                51381    513.701989
                51647    382.360458
                51912    263.747512
                52179    212.804529
                52445    193.571536
                52709    206.544407
                52976    231.809013
                53242    397.061661
                53507    271.110696
                53775    194.001184
                54040    186.844463
                54307    161.177186
                54571    151.644156
                54838    202.555505
                55104    369.126096
                55369    203.795961
                55637    145.588821
                55902    151.813262
                56168    144.833544
                56435    139.237035
                56701    181.181264
                56966    319.173606
                57233    207.718414
                57498    152.843207
       US       50889           NaN
      

In [37]:
#test_data

In [38]:
#pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

In [39]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [40]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'doubling_rate',
                            'level_2':'index'})
pd_DR_result.head()

,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1231,NaN


In [41]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,"Korea, South",1
2,2,2020-01-22,no,Kosovo,0
3,3,2020-01-22,no,Kuwait,0
4,4,2020-01-22,no,Kyrgyzstan,0


In [42]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index', 'doubling_rate']],on=['index'],how='left')

In [43]:
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN
2,2,2020-01-22,no,Kosovo,0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN


In [44]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply

In [45]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [46]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [47]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8
2,2,2020-01-22,no,Kosovo,0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0


# Filtered doubling rate

In [48]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
57717,no,Zimbabwe,56524,50.066843
57718,no,Zimbabwe,56722,59.212121
57719,no,Zimbabwe,57019,67.109339
57720,no,Zimbabwe,57254,64.986783
57721,no,Zimbabwe,57721,64.693291


In [49]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,confirmed_filtered_DR
57717,57717,2020-08-25,no,Barbados,164,108.000000,163.6,95.235294
57718,57718,2020-08-25,no,Belarus,70727,545.276705,70765.6,443.507538
57719,57719,2020-08-25,no,Belgium,82447,321.559035,82513.4,220.215550
57720,57720,2020-08-25,no,Albania,8759,51.789157,8759.0,53.403727
57721,57721,2020-08-25,no,Zimbabwe,6196,45.604010,6168.6,64.693291


In [50]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [51]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,confirmed_filtered_DR
56435,56435,2020-08-21,no,Germany,233029,139.237035,232476.4,170.778820
56701,56701,2020-08-22,no,Germany,233861,181.181264,233759.6,181.766380
56966,56966,2020-08-23,no,Germany,234494,319.173606,235017.8,183.954723
57233,57233,2020-08-24,no,Germany,236122,207.718414,236154.7,196.215078
57498,57498,2020-08-25,no,Germany,237583,152.843207,237291.6,207.718093


In [52]:
pd_result_larg.to_csv('C:/ProgramData/Anaconda3/eps_covid19/data/processed/COVID_final_set.csv',sep=';',index=False)